# Análise das Emendas PIX com Clusters

Este notebook carrega `data/dados_com_clusters.csv` e realiza as etapas descritas na metodologia do projeto:
1. *Exploração inicial*
2. *Modelagem OLS com efeitos fixos de partido*
3. *Modelagem multinível step-up*


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm


In [ ]:
df = pd.read_csv('../data/dados_com_clusters.csv').dropna()
df.head()

In [ ]:
sns.set(style='whitegrid')
plt.figure(figsize=(6,4))
sns.scatterplot(data=df,
                x='emendas_pix_per_capita_partido_prefeito_eleito',
                y='porcentagem_votos_validos_2024',
                alpha=0.5)
plt.xlabel('Emendas PIX per capita')
plt.ylabel('Votos válidos (proporção)')
plt.title('Distribuição dos dados')
plt.show()

In [ ]:
formula = 'porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito + C(sigla_partido_prefeito_eleito) + cluster_0 + cluster_1 + cluster_2 + cluster_3'
ols_res = smf.ols(formula, data=df).fit(cov_type='HC3')
print(ols_res.summary())

In [ ]:
# Modelo nulo
null_md = smf.mixedlm('porcentagem_votos_validos_2024 ~ 1', df, groups=df['sigla_partido_prefeito_eleito']).fit()

# Interceptos aleatórios
ri_md = smf.mixedlm('porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
                    df, groups=df['sigla_partido_prefeito_eleito']).fit()

# Intercepto e inclinação aleatórios
ris_md = smf.mixedlm('porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
                     df, groups=df['sigla_partido_prefeito_eleito'],
                     re_formula='~emendas_pix_per_capita_partido_prefeito_eleito').fit()

# Modelo completo com clusters
full_md = smf.mixedlm('porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito + cluster_0 + cluster_1 + cluster_2 + cluster_3',
                      df, groups=df['sigla_partido_prefeito_eleito'],
                      re_formula='~emendas_pix_per_capita_partido_prefeito_eleito').fit()

for lbl, m in [('Nulo', null_md), ('RI', ri_md), ('RI+RS', ris_md), ('Completo', full_md)]:
    print('
', lbl)
    display(m.summary())